# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/1f5yE/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298417969425                   -0.85    5.8         
  2   -8.300179712067       -2.75       -1.25    1.0    126ms
  3   -8.300428443787       -3.60       -1.86    1.8    191ms
  4   -8.300461091864       -4.49       -2.71    2.6    155ms
  5   -8.300464277566       -5.50       -3.19    2.6    174ms
  6   -8.300464444256       -6.78       -3.32    8.9    278ms
  7   -8.300464531577       -7.06       -3.46    2.4    187ms
  8   -8.300464591116       -7.23       -3.61    1.4    186ms
  9   -8.300464632558       -7.38       -3.84    1.6    146ms
 10   -8.300464638159       -8.25       -3.95    1.4    136ms
 11   -8.300464643512       -8.27       -4.28    1.9    159ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67450623376                   -0.70    5.6         
  2   -16.67870766462       -2.38       -1.14    1.6    360ms
  3   -16.67921251775       -3.30       -1.87    2.0    400ms
  4   -16.67927805733       -4.18       -2.76    5.6    493ms
  5   -16.67928595998       -5.10       -3.19    4.0    494ms
  6   -16.67928617999       -6.66       -3.47    2.6    419ms
  7   -16.67928621324       -7.48       -3.91    2.4    385ms
  8   -16.67928622137       -8.09       -4.53    2.4    404ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32544858255                   -0.56    7.2         
  2   -33.33274716594       -2.14       -1.00    1.4    1.29s
  3   -33.33406482439       -2.88       -1.73    4.5    1.60s
  4   -33.33427870953       -3.67       -2.61    2.5    1.41s
  5   -33.33694217502       -2.57       -2.50   11.8    2.29s
  6   -33.33694256176       -6.41       -2.46    2.5    1.33s
  7   -33.33694187849   +   -6.17       -3.13    1.0    1.14s
  8   -33.33694305320       -5.93       -3.44    5.9    1.63s
  9   -33.33694372134       -6.18       -4.11    2.4    1.35s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298550622170                   -0.85    5.0         
  2   -8.300278890314       -2.76       -1.58    1.0    110ms
  3   -8.300382860048       -3.98       -2.31    2.9    181ms
  4   -8.300326811505   +   -4.25       -2.18    3.4    177ms
  5   -8.300463983022       -3.86       -3.47    1.0    119ms
  6   -8.300464554559       -6.24       -3.75    3.4    202ms
  7   -8.300464637915       -7.08       -4.20    1.0    125ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32571332558                   -0.56    7.2         
  2   -33.32563078328   +   -4.08       -1.27    1.0    1.03s
  3   -22.79883436483   +    1.02       -0.56    6.1    2.08s
  4   -33.27804093104        1.02       -1.60    6.5    2.20s
  5   -33.14932851718   +   -0.89       -1.27    2.9    1.55s
  6   -31.97600296192   +    0.07       -1.00    4.6    1.78s
  7   -33.33112176306        0.13       -2.07    4.1    1.64s
  8   -33.33339007363       -2.64       -2.07    2.4    1.48s
  9   -33.33625721503       -2.54       -2.41    1.8    1.27s
 10   -33.33630076080       -4.36       -2.62    1.9    1.27s
 11   -33.33687514582       -3.24       -3.06    3.0    1.30s
 12   -33.33692480775       -4.30      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.